## Practica 1 Stadistical Learning 2


**21000341 - Jherson Sazo**

Descripción:

Llenar información de estatura y edad en la pestaña normales(por favor llenar hoy mismo):  
Datos
* Utilizar estos datos(primera pestaña) para implementar :
    * Hard-clustering con k-means (no usar sklearn)
    * Soft-clustering con GMM usando sklearn(sklearn.mixture.GaussianMixture)
* Ejecutar al menos 5 experimentos para seleccionar el valor de “K”
    * Ambos métodos pueden usar el mismo valor de “K”
    * Analizar el centroide de cada cluster y determinar si es posible asignar una categoría  cada cluster
* Analizar y concluir si el método del codo es adecuado para este problema
* Usar los datos en la segunda pestaña(valtest(normales)) y estimar:
    * El cluster que k-means asigna cada uno.
    * La probabilidad de pertenecer a cada cluster según GMM.
* Usar GMM y simular 1000 observaciones para estimar(estimación de Monte Carlo) el valor esperado(promedio) de la función f   f(edad, estatura) = estatura /edad

In [2]:
import numpy as np # Import the necessary packages.
import pandas as pd 
import matplotlib.pyplot as plt

In [4]:
#cluster = pd.read_csv('k-means clustering.csv') # Read data file into 'cluster'
    #cluster = open_excel_by_sheetname('estaturas.xlsx',0) # Read data file into 'cluster'
    #cluster.columns = ['x','y']
data = open_excel_by_sheetname('estaturas.xlsx',0)
data.columns = ['x','y']

In [29]:
def train_k_means(cluster,k,number_of_iterations):
    
    rows = cluster.shape[0] # 'rows' contains the total number of rows in cluster data.
    cols = cluster.shape[1] # 'cols' contains the total number of columns in cluster data.
  
    k_initializer = np.random.randint(1,rows,k)
    centroids = cluster.loc[k_initializer] # Randomly initialises 'k' no. of centroids.

    centroids['new'] = list(range(1,k+1)) # New indices 1 to k are set for the dataframe 'centroids'.
    centroids.set_index('new',inplace = True) 
    d = np.random.rand(rows) # Initialization of 'd' which would contain the centroid number closest to data point.

    #number_of_iterations = 15
    temp_epsilon = list(range(number_of_iterations)) # 'temp_epsilon' is the sum of squares of distances between points and centroid of a cluster for each iteration.

    for i in range(0,number_of_iterations): # This 'for' loop is for iterations.

        for j in range(0,rows): # This 'for' loop finds the centroid number closest to the data point.
            d[j] = ((centroids - cluster.loc[j])**2).sum(axis = 1).idxmin()
        cluster['centroid number'] = d # A new column 'centroid number' is added to dataframe 'cluster'.

        mean_x = list(range(k)) # Initialisation of 'mean_x' which will store mean of 'x' values of each cluster.
        mean_y = list(range(k)) # Initialisation of 'mean_y' which will store mean of 'y' values of each cluster.
        for m in range(0,k): # This 'for' loop calculates mean of 'x' and 'y' values of each cluster.
            mean_x[m] = cluster[cluster['centroid number'] == (m+1)]['x'].mean()
            mean_y[m] = cluster[cluster['centroid number'] == (m+1)]['y'].mean()
        centroids.replace(list(centroids['x']),mean_x,inplace = True) # The 'centroids' are replaced with the new values.
        centroids.replace(list(centroids['y']),mean_y,inplace = True)
    
        z = list(range(k)) # Initialisation of z  and centroid of each cluster.
        for p in range(0,k): # This 'for' loop calculates square of distances between data points and centroid of each cluster.
            z[p] = ((cluster[cluster['centroid number'] == p+1][['x','y']] - centroids.iloc[p])**2).values.sum()
        temp_epsilon[i] = round(sum(z),4) 
        
        if i > 0 and temp_epsilon[i-1] == temp_epsilon[i]:
            print("SON los costos iguales??",temp_epsilon[i-1],temp_epsilon[i])
            return [cluster,centroids,temp_epsilon[i]]
    print("LLEGO AL FINAL")
    return [cluster,centroids,temp_epsilon[-1]]

In [17]:
def open_excel_by_sheetname(full_path, name_sheet):
    try:
        df = pd.read_excel(full_path, sheet_name=name_sheet)
        return df
    except Exception as error:
        raise error

In [32]:
cluster = open_excel_by_sheetname('estaturas.xlsx',0) # Read data file into 'cluster'
cluster.head()
cluster.columns = ['x','y']
print(train_k_means(cluster,4,10))

SON los costos iguales?? 189.1207 189.1207
[       x     y  centroid number
0   1.77  26.0              1.0
1   1.74  31.0              2.0
2   1.72  24.0              3.0
3   1.78  34.0              2.0
4   1.65  32.0              2.0
5   1.66  29.0              1.0
6   1.64  27.0              1.0
7   1.85  34.0              2.0
8   1.85  26.0              1.0
9   1.75  21.0              3.0
10  1.73  23.0              3.0
11  1.79  25.0              3.0
12  1.82  24.0              3.0
13  1.65  25.0              3.0
14  1.79  25.0              3.0
15  1.72  34.0              2.0
16  1.70  26.0              1.0
17  1.57  25.0              3.0
18  1.66  26.0              1.0
19  1.75  27.0              1.0
20  1.77  32.0              2.0
21  1.80  40.0              4.0
22  1.85  25.0              3.0
23  1.82  24.0              3.0
24  1.75  38.0              2.0
25  1.73  34.0              2.0
26  1.55  29.0              1.0
27  1.76  28.0              1.0
28  1.68  27.0              

**Nota**:
Haciendo varios experimentos, me he podido dar cuenta que el valor de costo varia en cada prueba con el mismo numero de K, por lo que he desarrollado un metodo para hacer iterativo a un numero X de iteraciones con el mismo valor de K, buscando obtener el costo mas bajo, y asi tener mejor los centroides.

La funcion codo, es iterar variando el numero de K para obtener los centroides con el costo mas bajo, pero esta funcionalidad se va ver reflejada en el siguiente metodo probando 5 valores de K, adicionando lo mencionado anteriormente.

In [ ]:
def test_k_means(reps,k_list,cluster,iteraciones):
    costs = {}
    for k in k_list:
        for i in range(reps):
            results = train_k_means(cluster,k,iteraciones)
            costs[results[2]]= results
    
            
        

## Soft-clustering con GMM

In [8]:
from sklearn.mixture import GaussianMixture

In [9]:
def train_GMM(data, K):
    gm = GaussianMixture(n_components=K, random_state=0).fit(data)
    gm.means_
    return gm


In [24]:
#Experimento 1, 2 clusters
gm = train_GMM(cluster,2)
print(gm.means_)
print(gm.predict([[1.70,28]]))

[[ 1.72384042 25.89825759]
 [ 1.74628242 35.17238953]]
[0]


In [32]:
#Experimento 2, 3 clusters
gm = train_GMM(cluster,3)
print(gm.means_)
print(gm.predict([[1.70,28]]))

[[ 1.72807829 25.58286677]
 [ 1.77666741 48.33323415]
 [ 1.73243803 33.4444868 ]]
[0]


In [26]:
#Experimento 3, 4 clusters
gm = train_GMM(cluster,4)
print(gm.means_)
print(gm.predict([[1.70,28]]))

[[ 1.67056857 28.20035884]
 [ 1.77666859 48.33329889]
 [ 1.75158586 34.17089106]
 [ 1.7503387  24.65611208]]
[0]


In [30]:
#Experimento 4, 5 clusters
gm = train_GMM(cluster,5)
print(gm.means_)
print(gm.predict([[1.70,28]]))

[[ 1.67897568 27.8123578 ]
 [ 1.84       51.        ]
 [ 1.73048713 32.97000029]
 [ 1.7478479  24.64599493]
 [ 1.73406862 40.29442874]]
[0]


In [28]:
#Experimento 5, 1 clusters
gm = train_GMM(cluster,1)
print(gm.means_)
print(gm.predict([[1.70,28]]))

[[ 1.7326087  29.52173913]]
[0]


**Calulando las predicciones del excel**

In [38]:
test = open_excel_by_sheetname('estaturas.xlsx',1) # Read data file into 'cluster'
print(test.head(7))
predicts = gm.predict(test)
print(predicts)



   Estatura(metros)  Edad(años)
0              1.82        25.0
1              1.80        27.0
2              1.60        31.0
3              1.60        35.0
4              1.82        30.0
5              1.76        32.0
6              1.79        31.0
[0 0 2 2 0 2 2 2]


**Estimar el metodo MonteCarlo con 1000 observaciones**

In [47]:
#array de estaturas
height = np.round(np.random.uniform(low=1.4, high=2.01, size=(1000,)),2)
age = np.round(np.random.uniform(low=12, high=28, size=(1000,)),0)



[27. 24. 14. 16. 22. 22. 28. 15. 13. 16. 27. 15. 15. 15. 16. 13. 13. 13.
 17. 17. 26. 21. 16. 16. 15. 15. 16. 22. 16. 24. 18. 17. 21. 19. 20. 21.
 13. 12. 28. 24. 25. 27. 21. 27. 23. 13. 27. 24. 24. 18. 15. 15. 15. 16.
 20. 15. 22. 26. 24. 15. 14. 16. 19. 24. 21. 13. 22. 24. 15. 15. 17. 22.
 25. 14. 21. 22. 25. 18. 13. 20. 25. 25. 21. 20. 20. 16. 17. 13. 15. 21.
 16. 27. 27. 23. 26. 15. 20. 21. 17. 22. 15. 13. 20. 17. 16. 13. 15. 24.
 25. 26. 23. 21. 16. 23. 23. 16. 15. 26. 23. 20. 17. 27. 23. 15. 20. 25.
 23. 19. 16. 27. 25. 23. 18. 26. 25. 17. 17. 25. 21. 27. 26. 24. 16. 20.
 14. 21. 14. 18. 19. 20. 15. 22. 24. 13. 14. 17. 21. 15. 25. 12. 14. 26.
 18. 13. 16. 14. 24. 17. 20. 21. 22. 22. 25. 25. 19. 25. 20. 13. 15. 27.
 20. 26. 19. 18. 24. 18. 19. 18. 14. 24. 18. 16. 22. 27. 13. 20. 12. 21.
 23. 13. 23. 25. 13. 20. 15. 13. 27. 24. 25. 27. 16. 15. 27. 14. 25. 26.
 17. 21. 14. 16. 24. 19. 24. 26. 22. 20. 16. 27. 19. 26. 17. 24. 14. 24.
 20. 26. 14. 24. 21. 24. 18. 26. 22. 27. 14. 17. 13

In [8]:
np.random.randint(7,15+1,4)

array([12, 15, 15, 10])

In [13]:
d = np.random.rand(27)
temp_epsilon = list(range(15))
temp_epsilon

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]